In [3]:
import keras
import tensorflow as tf
import sys
import os
print('TensorFlow version:', tf.__version__)
print('Keras version:', keras.__version__)

Using TensorFlow backend.


TensorFlow version: 1.8.0
Keras version: 2.2.2


In [7]:
DEVICE = "/gpu:0" 

working_dir = os.getcwd()
print("Working directory is:",working_dir)
sys.path.append(working_dir)  # To find local version of the library

#ROOT_DIR = os.path.join("/home/user/Source/Evrim/Turkishplate") #linux
ROOT_DIR = os.path.join("C:/Users/is95217/source/turkishplate") #win
sys.path.append(ROOT_DIR + "/Python")
import Modelop as p

#DATA_DIR = "/home/user/Source/data/dataEvrim/Original_15Kcrop_distributed"  #linux
DATA_DIR = "C:/Users/is95217/data/plaka_out_distr/" #win
print("Data:", DATA_DIR)
print("Source code:", ROOT_DIR)

Working directory is: C:\Users\is95217\source\turkishplate
Data: C:/Users/is95217/data/plaka_out_distr/
Source code: C:/Users/is95217/source/turkishplate


# Model description and training

In [8]:
#                imgw,imgh,path,load, path & name_model, epoch_cnt
model2 = p.train2gru(128,64,DATA_DIR,False, ROOT_DIR + "/Model/model_deneme", 1)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 64, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 64, 32, 16)   0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 32, 16)   2320        max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

AttributeError: 'Model' object has no attribute 'add'

In [ ]:

model2.inputs 
 #   p.layer_to_visualize(p.gru1_merged,model2)

In [ ]:
model2.summary

In [ ]:
import matplotlib.pyplot as plt
# Get training and test loss histories
training_loss = model2.history.history['loss']
test_loss = model2.history.history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();


In [ ]:
training_acc = model2.history.history['acc']
test_acc = model2.history.history['val_acc']
# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training acc', 'Test acc'])
plt.xlabel('Epoch')
plt.ylabel('acc')
plt.show();

# Test on validation images

In [ ]:
cnt_test = len(os.listdir(DATA_DIR + '/test'))

In [ ]:
tiger_test = p.TextImageGenerator(DATA_DIR, 'test', 128, 64, cnt_test, 4)
tiger_test.build_data()

net_inp = model2.get_layer(name='the_input').input
net_out = model2.get_layer(name='softmax').output

from keras import backend as K
sess = tf.Session()
K.set_session(sess)


In [ ]:
cnt_test= len(os.listdir(DATA_DIR + '/test'))
img_filepath = DATA_DIR + '/test/' 

In [ ]:
matched=0
start='#'
end='#'
for fname in os.listdir(DATA_DIR + '/test'):
     
    image = cv2.imread(img_filepath + fname)
    if p.check_word(fname):                
        fn= (fname.split(start))[1].split(end)[0] 
    else:
        fn = fname.split('.')   
        fn= fn[0]
        
    with graph_detection.as_default():
         with session_detection.as_default():
            predicted_text = p.predict_single_plate(session_detection,loaded_model,image)#input directory of pic
            
    
    
    if ''.join(predicted_text) == fn:
        matched +=1
    else:
        print('notmatched',''.join(predicted_text),'-',fn)
        #print('matched')